# **ANN arch**

In [ ]:
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using:{device}')

using:cuda


In [ ]:
!unzip "/content/fashion-mnist_train.csv.zip" -d "/content"

Archive:  /content/fashion-mnist_train.csv.zip
  inflating: /content/fashion-mnist_train.csv  


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 22.1 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
df = pd.read_csv("/content/fashion-mnist_train.csv")

In [ ]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
type(X)

numpy.ndarray

In [ ]:
X_train_np, X_test_np, y_train_np,  y_test_np = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_np = X_train_np/255.0
X_test_np = X_test_np/255.0

In [ ]:
X_train = torch.from_numpy(X_train_np).to(dtype= torch.float32)
y_train = torch.from_numpy(y_train_np).to(dtype = torch.long)
X_test = torch.from_numpy(X_test_np).to(dtype= torch.float32)
y_test = torch.from_numpy(y_test_np).to(dtype = torch.long)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train_data = TensorDataset(X_train_np, y_train_np)
test_data = TensorDataset(X_test_np, y_test_np)

In [ ]:
batch_size = 64

In [ ]:
train_Loder = DataLoader(train_data, batch_size= batch_size, shuffle=True)
test_Loder = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
class MY_model(nn.Module):

  def __init__(self, fetures):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(fetures, 192),
        nn.ReLU(),
        nn.Droupout(0.3),
        nn.Linear(192, 256),
        nn.ReLU(),
        nn.Droupout(0.2),
        nn.Linear(64, 10)
    )

  def forward(self, X):
    return self.model(X)

In [ ]:
model = MY_model(X_train.shape[1]).to(device)

In [ ]:
lr = 0.1
epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

In [ ]:
model.train()
for epoch in range(epochs):
  total_loss = 0
  for batch_features, batch_labels in train_Loder:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    logist = model(batch_features)
    loss = criterion(logist, batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  avg_loss = total_loss/len(train_Loder)
  print(f'epoch :{epoch +1} loss : {avg_loss}')


epoch :1 loss : 0.5520416678190231
epoch :2 loss : 0.36683112700780235
epoch :3 loss : 0.3152758907179038
epoch :4 loss : 0.28567005427678427
epoch :5 loss : 0.26120245349407195
epoch :6 loss : 0.24688619753221672
epoch :7 loss : 0.22751775560776394
epoch :8 loss : 0.21157034164170424
epoch :9 loss : 0.19941512541969617
epoch :10 loss : 0.19119276695946852
epoch :11 loss : 0.17866950306296348
epoch :12 loss : 0.1703589951346318
epoch :13 loss : 0.1662338675558567
epoch :14 loss : 0.15797316977878412
epoch :15 loss : 0.14893953812370697
epoch :16 loss : 0.1406944880783558
epoch :17 loss : 0.135050387814641
epoch :18 loss : 0.1314353955425322
epoch :19 loss : 0.12421898386316994
epoch :20 loss : 0.11810431931416193
epoch :21 loss : 0.11584915568431219
epoch :22 loss : 0.11082605572417378
epoch :23 loss : 0.10903052110162874
epoch :24 loss : 0.10310673272858063
epoch :25 loss : 0.09444742785642544
epoch :26 loss : 0.09474585125781596
epoch :27 loss : 0.09218672208612164
epoch :28 loss : 0

In [ ]:
model.eval()
total = 0
correct =0
with torch.no_grad():
  for batch_features, batch_labels in test_Loder:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    logist = model(batch_features)
    _, predict = torch.max(logist, 1)
    total = total + batch_labels.shape[0]
    correct = correct + (predict == batch_labels).sum().item()
  accuracy = correct/ total
print(accuracy)



0.9216666666666666


In [ ]:
total_train =0
correct_train =0
with torch.no_grad():
  for batch_features, batch_labels in train_Loder:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    logist = model(batch_features)
    _, predict = torch.max(logist, 1)
    total = total + batch_labels.shape[0]
    correct = correct + (predict == batch_labels).sum().item()
    accuracy = correct/ total
print(accuracy)

0.97955


In [ ]:
def def_model(trial):
  n_layers = trial.suggest_int('n_layers', 1 ,5)
  layers = []

  input_size = X_train.shape[1]
  for i in range(n_layers):
    output_size = trial.suggest_int(f'n_nuron {i}', 32,256, 32)
    layers.append(torch.nn.Linear(input_size, output_size))
    layers.append(torch.nn.ReLU(inplace=True))
    p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
    layers.append(torch.nn.Dropout(p))
    input_size = output_size
  layers.append(torch.nn.Linear(input_size, 10))

  model = torch.nn.Sequential(*layers).to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)


  model.train()
  tuning_epoch = 5
  for epoch in range(epochs):
    total_loss = 0
    for batch_features, batch_labels in train_Loder:
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
      logist = model(batch_features)
      loss = criterion(logist, batch_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    avg_loss = total_loss/len(train_Loder)
    # print(f'epoch :{epoch +1} loss : {avg_loss}')

  model.eval()
  total = 0
  correct =0
  with torch.no_grad():
    for batch_features, batch_labels in test_Loder:
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
      logist = model(batch_features)
      _, predict = torch.max(logist, 1)
      total = total + batch_labels.shape[0]
      correct = correct + (predict == batch_labels).sum().item()
    accuracy = correct/ total
    # print(accuracy)
  return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(def_model, n_trials= 20)

[I 2025-06-05 08:42:35,034] A new study created in memory with name: no-name-9ba297e3-d8e6-4282-ba55-01edddb940b0
<ipython-input-57-16bf9c5daa29>:7: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  output_size = trial.suggest_int(f'n_nuron {i}', 32,256, 32)
[I 2025-06-05 08:45:49,913] Trial 0 finished with value: 0.8859166666666667 and parameters: {'n_layers': 4, 'n_nuron 0': 64, 'dropout_l0': 0.20151616162722016, 'n_nuron 1': 192, 'dropout_l1': 0.3760427781798782, 'n_nuron 2': 160, 'dropout_l2': 0.24490265503569317, 'n_nuron 3': 128, 'dropout_l3': 0.309041701811585}. Best is trial 0 with value: 0.8859166666666667.
[I 2025-06-05 08:49:23,900] Trial 1 finished with value: 0.8716666666666667 and parameters: {'n_layers': 5, 'n_nuron 0': 256, 'dropout_l0': 0.30371130784797634, 'n_nuron 1': 32, 'dropout_l1': 0.499493832249112, 'n_nuron 2': 32, 'dropout_l2': 0.2908319695632854, 'n_nuron 3': 64, 'dropout_l3': 0.289089

In [ ]:
study.best_value

0.8988333333333334

In [ ]:
study.best_params

{'n_layers': 2,
 'n_nuron 0': 192,
 'dropout_l0': 0.3347515678985543,
 'n_nuron 1': 256,
 'dropout_l1': 0.2700810583994086}